# Title: ...

## 0. Imports

In [1]:
# Notebook similar to: https://www.kaggle.com/code/gracehephzibahm/prediction-of-rent-prices-in-barcelona
# Web scraper with good insights and EDA: https://github.com/agonzalezramos/Idealista-Price-Prediction/tree/main

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd

In [3]:
""" archivo = pd.read_csv('archivo.csv', sep=',')
# Split archivo.csv into 4 parts
archivo1 = archivo.iloc[:2000]
archivo2 = archivo.iloc[2001:4000]
archivo3 = archivo.iloc[4001:6000]
archivo4 = archivo.iloc[6001:]

archivo1.to_csv('archivo1.csv', sep=',')
archivo2.to_csv('archivo2.csv', sep=',')
archivo3.to_csv('archivo3.csv', sep=',')
archivo4.to_csv('archivo4.csv', sep=',') """

" archivo = pd.read_csv('archivo.csv', sep=',')\n# Split archivo.csv into 4 parts\narchivo1 = archivo.iloc[:2000]\narchivo2 = archivo.iloc[2001:4000]\narchivo3 = archivo.iloc[4001:6000]\narchivo4 = archivo.iloc[6001:]\n\narchivo1.to_csv('archivo1.csv', sep=',')\narchivo2.to_csv('archivo2.csv', sep=',')\narchivo3.to_csv('archivo3.csv', sep=',')\narchivo4.to_csv('archivo4.csv', sep=',') "

## 1. Load Data

In [4]:
# Load Data
og_df = pd.read_csv('../data/pisos.csv', sep=',')

C:\Users\pdpau\AppData\Local\Temp\ipykernel_21140\3555031827.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  og_df = pd.read_csv('../data/pisos.csv', sep=',')


## 2. Data Preparation

In [5]:
df = og_df.copy()

df.head(5)

,Unnamed: 0,summary,description,location,photo,recomendado,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
0,0,Casa en calle Urb. Las Mimosas,NaN,Ames (San Tome),https://fotos4.imghs.net/nrd/1069/104/1069_I00...,Recomendado,250.000 €,315 m²,4,793 €/m²,3,31,pisos,a_coruna
1,1,Piso en Milladoiro,NaN,Ames (San Tome),https://fotos4.imghs.net/nrd/1069/113/1069_I00...,Recomendado,90.000 €,72 m²,2,1.250 €/m²,1,11,pisos,a_coruna
2,2,Casa en calle La Piedra,NaN,A Ortigueira (Cariño),https://fotos1.imghs.net/nrd//999170/892070610...,Recomendado,50.000 €,92 m²,2,543 €/m²,1,40,pisos,a_coruna
3,3,Casa en Perillo,NaN,Perillo (Oleiros),https://fotos3.imghs.net/nrd/1089/198/1089_XA2...,Recomendado,359.000 €,244 m²,4,1.471 €/m²,3,39,pisos,a_coruna
4,4,Chalet en Urbanización Las Mimosas,NaN,O Milladoiro (Ames),https://fotos5.imghs.net/nrd/1097/754/1097_0_f...,Recomendado,200.430 €,315 m²,4,636 €/m²,3,24,pisos,a_coruna


In [6]:
# Drop innecesary columns
df.drop(columns=["Unnamed: 0", "description", "photo", "recomendado"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777179 entries, 0 to 777178
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   summary     777179 non-null  object
 1   location    776886 non-null  object
 2   price       776898 non-null  object
 3   size        776892 non-null  object
 4   rooms       752352 non-null  object
 5   price/m2    744608 non-null  object
 6   bathrooms   752526 non-null  object
 7   Num Photos  776898 non-null  object
 8   type        776898 non-null  object
 9   region      776282 non-null  object
dtypes: object(10)
memory usage: 59.3+ MB


In [7]:
# Drop NaN values
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 719459 entries, 0 to 777178
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   summary     719459 non-null  object
 1   location    719459 non-null  object
 2   price       719459 non-null  object
 3   size        719459 non-null  object
 4   rooms       719459 non-null  object
 5   price/m2    719459 non-null  object
 6   bathrooms   719459 non-null  object
 7   Num Photos  719459 non-null  object
 8   type        719459 non-null  object
 9   region      719459 non-null  object
dtypes: object(10)
memory usage: 60.4+ MB


## 3. Exploratory Data Analysis

In [8]:
df.tail(5)

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
777170,"Finca rústica en Carrer Bosc, nº 51",Lliçà d'Amunt,1.100.000 €,1.120 m²,10,982 €/m²,1.0,32,fincas_rusticas,valles_oriental_sud
777171,Finca rústica en Els Dominics,Cardedeu,1.500.000 €,520 m²,8,2.884 €/m²,3.0,12,fincas_rusticas,valles_oriental_sud
777176,Finca rústica en La Roca del Vallès,La Roca del Vallès,1.350.000 €,350 m²,5,3.857 €/m²,2.0,9,fincas_rusticas,valles_oriental_sud
777177,"Finca rústica en Bajada Les Valls, nº S/N",La Roca del Vallès,456.000 €,202 m²,5,2.257 €/m²,3.0,30,fincas_rusticas,valles_oriental_sud
777178,Loft en calle Catlunya,Llerona (Les Franqueses del Vallès),155.000 €,87 m²,1,1.781 €/m²,1.0,34,lofts,valles_oriental_sud


### 3.1 Dataset compraventa

In [9]:
# TODO: Explore by location

In [10]:
# Filtrar nomes per Catalunya

# value counts de region
df['region'].value_counts()

# Filtrar nomes per Barcelona
df_barna = df[df['region'] == 'barcelona']
df_barna

,summary,location,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
20983,"Piso en Carrer del Bruc, cerca de Carretera de...",Valldaura-Plaça Catalunya (Manresa),85.000 €,87 m²,4,977 €/m²,1,10,pisos,barcelona
20984,"Piso en Carrer de les Orenetes, 37",Viladecavalls,149.900 €,57 m²,2,2.629 €/m²,1,16,pisos,barcelona
20985,"Casa en Carrer de la Creu, cerca de Carrer de ...",Sant Just Desvern,1.095.000 €,371 m²,6,2.951 €/m²,4,46,pisos,barcelona
20986,Piso en Centre-Zona Estació-Passeig de Joan Ma...,Centre-Zona Estació-Passeig de Joan Maragall (...,297.000 €,112 m²,3,2.651 €/m²,2,30,pisos,barcelona
20987,"Piso en Carrer del Retir, 4",Castellar del Vallès,230.000 €,70 m²,3,3.285 €/m²,1,1,pisos,barcelona
...,...,...,...,...,...,...,...,...,...,...
269325,"Loft en Carrer de Francisco Giner, nº 40",La Vila de Gràcia (Distrito Gràcia. Barcelona ...,380.000 €,180 m²,2,2.111 €/m²,2.0,11,lofts,barcelona
269326,"Loft en Carrer de Ramón y Cajal, 58",La Vila de Gràcia (Distrito Gràcia. Barcelona ...,220.000 €,87 m²,3,2.528 €/m²,1.0,8,lofts,barcelona
269327,"Loft en Carrer d'Andrade, nº 218",Sant Martí de Provençals (Distrito Sant Martí....,113.000 €,55 m²,1,2.054 €/m²,1.0,9,lofts,barcelona
269330,Loft en Barcelona,Les Corts (Distrito Les Corts. Barcelona Capital),419.300 €,93 m²,1,4.508 €/m²,1.0,20,lofts,barcelona


In [ ]:
# Filtrar si location contiene Manresa
df_manresa = df_barna[df_barna['location'].str.contains('Sabadell')]
df_manresa['location'].value_counts()

location
Centre (Distrito Centre-Covadonga. Sabadell)                                        91
La Creu Alta (Sabadell)                                                             77
Can Rull (Sabadell)                                                                 56
Gràcia (Sabadell)                                                                   42
Ca n'Oriac (Sabadell)                                                               35
La Creu de Barberà (Sabadell)                                                       35
Eixample-Sant Oleguer (Sabadell)                                                    35
Les Termes (Sabadell)                                                               19
La Concòrdia (Sabadell)                                                             16
La Serra d'en Cameró (Distrito Can Feu-La Serra d'en Cameró. Sabadell)              14
Can Puiggener (Distrito Can Puiggener-El Torrent del Capellà. Sabadell)             12
Can Deu (Distrito Can Deu-La Plana

### 3.2 Index preu dels lloguers per municipi 2022 (es podria fer per districte de Barna)

In [12]:
# Fuente: https://www.mivau.gob.es/vivienda/alquila-bien-es-tu-derecho/serpavi

# Carga el archivo SHP
shapefile = gpd.read_file("../data/ALQ_distritos/ALQ_Distritos_2022_Web.shp")

# Guarda los datos en un archivo CSV
shapefile.to_csv("distritos.csv", index=False)


In [14]:
pd.set_option('display.max_columns', None)

In [18]:
""" a1 = pd.read_csv("archivo1.csv", sep=',')
a2 = pd.read_csv("archivo2.csv", sep=',')
a3 = pd.read_csv("archivo3.csv", sep=',')
a4 = pd.read_csv("archivo4.csv", sep=',')

new_df = pd.concat([a1, a2, a3, a4], ignore_index=True) """

new_df = pd.read_csv("distritos.csv", sep=',')
#new_df.dropna(inplace=True)
new_df[1000:1010]

,CUMUN,CDIS,CMUN,CPRO,CCA,CUDIS,OBS,CNUT0,CNUT1,CNUT2,CNUT3,CLAU2,NPRO,NCA,NMUN,OBJECTID,CPRO_1,LITPRO,CUMUN_1,LITMUN,CUDIS_1,TITULO_VC,Num_VC,Renta_Medi,Renta_Perc,Renta_Pe_1,Cuantía_m,Cuantia_me,Cuantia__1,Superficie,Superfic_1,Superfic_2,TITULO_VU,Num_VU,Renta_Me_1,Renta_Pe_2,Renta_Pe_3,Cuantía_1,Cuantia__2,Cuantia__3,Superfic_3,Superfic_4,Superfic_5,geometry
1000,43062,1,62,43,9,4306201,NaN,ES,5,1,4,43062,Tarragona,Cataluña,Freginals,33742,43.0,Tarragona,43062.0,Freginals,4306201.0,COLECTIVA,0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,UNIFAMILIAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((57978.76649999991 4965559.954000004,..."
1001,45072,1,72,45,8,4507201,NaN,ES,4,2,5,45072,Toledo,Castilla-La Mancha,"Herencias, Las",33743,45.0,Toledo,45072.0,"Herencias, Las",4507201.0,COLECTIVA,0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,UNIFAMILIAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLYGON ((-530098.5637999997 4855533.386399999...
1002,41037,2,37,41,1,4103702,NaN,ES,6,1,8,41037,Sevilla,Andalucía,"Corrales, Los",33744,41.0,Sevilla,41037.0,"Corrales, Los",4103702.0,COLECTIVA,0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,UNIFAMILIAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLYGON ((-554725.1101000011 4452538.891400002...
1003,50131,1,131,50,2,5013101,NaN,ES,2,4,3,50131,Zaragoza,Aragón,Jaulín,33745,50.0,Zaragoza,50131.0,Jaulín,5013101.0,COLECTIVA,0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,UNIFAMILIAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLYGON ((-102959.96829999983 5075403.72320000...
1004,45190,1,190,45,8,4519001,NaN,ES,4,2,5,45190,Toledo,Castilla-La Mancha,Villaminaya,33746,45.0,Toledo,45190.0,Villaminaya,4519001.0,COLECTIVA,0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,UNIFAMILIAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLYGON ((-432602.9404999986 4827852.869099997...
1005,16055,1,55,16,8,1605501,NaN,ES,4,2,3,16055,Cuenca,Castilla-La Mancha,Carboneras de Guadazaón,33747,16.0,Cuenca,16055.0,Carboneras de Guadazaón,1605501.0,COLECTIVA,0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,UNIFAMILIAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLYGON ((-205078.8275000006 4862356.172200002...
1006,45022,1,22,45,8,4502201,NaN,ES,4,2,5,45022,Toledo,Castilla-La Mancha,Buenaventura,33748,45.0,Toledo,45022.0,Buenaventura,4502201.0,COLECTIVA,0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,UNIFAMILIAR,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLYGON ((-539841.6019000001 4893865.798299998...
1007,37274,2,274,37,7,3727402,NaN,ES,4,1,5,37274,Salamanca,Castilla y León,Salamanca,33749,37.0,Salamanca,37274.0,Salamanca,3727402.0,COLECTIVA,1838,5.430696,4.500000,6.627895,491.834958,405.00000,577.363636,92.0,76.0,108.0,UNIFAMILIAR,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-625907.8942999989 5007775.0145, -62..."
1008,41091,11,91,41,1,4109111,NaN,ES,6,1,8,41091,Sevilla,Andalucía,Sevilla,33750,41.0,Sevilla,41091.0,Sevilla,4109111.0,COLECTIVA,1585,8.279570,6.760731,10.191515,750.000000,600.00000,916.290161,93.0,67.0,121.0,UNIFAMILIAR,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MULTIPOLYGON (((-667512.6358999982 4492311.931...
1009,37274,1,274,37,7,3727401,NaN,ES,4,1,5,37274,Salamanca,Castilla y León,Salamanca,33751,37.0,Salamanca,37274.0,Salamanca,3727401.0,COLECTIVA,1527,7.368421,5.769231,9.150997,571.824129,459.42345,777.990833,85.0,60.0,113.0,UNIFAMILIAR,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POLYGON ((-630635.9701999985 5007895.059500001...


In [39]:
catalunya_df = new_df.copy()

provincies_cat = ['Barcelona', 'Girona', 'Lleida', 'Tarragona']

catalunya_df = catalunya_df[catalunya_df['LITPRO'].isin(provincies_cat)]

""" new_df['renta_med_total_vc'] = new_df['renta_med_por_m2_vc'] * new_df['superficie_med_vc']
new_df['renta_med_total_vu'] = new_df['renta_med_por_m2_vu'] * new_df['superficie_med_vu'] """

catalunya_df = catalunya_df.drop(columns=['CCA', 'OBS', 'CNUT0', 'CNUT1', 'CNUT2', 'CNUT3', 'OBJECTID', 'CPRO_1', 'CUMUN_1', 'TITULO_VC', 'TITULO_VU', 'LITMUN', 'LITPRO'])

catalunya_df = catalunya_df[['CUMUN', 'CDIS', 'CMUN', 'CPRO', 'CUDIS', 'CLAU2', 'NPRO', 'NCA', 'NMUN', 'Num_VC', 'Renta_Medi', 'Cuantía_m', 'Superficie', 'Num_VU', 'Renta_Me_1', 'Cuantía_1', 'Superfic_3']]

catalunya_df = catalunya_df.rename(columns={
    'NAMEUNIT': 'municipio', 'Num_VC': 'num_testigos_vc', 'Renta_Medi': 'renta_med_por_m2_vc', 'Cuantía_m': 'precio_mes_med_vc', 'Superficie': 'superficie_med_vc', 
    'Num_VU': 'num_testigos_vu', 'Renta_Me_1': 'renta_med_por_m2_vu', 'Cuantía_1': 'precio_mes_med_vu', 'Superfic_3': 'superficie_med_vu'
})

In [41]:
districtes_BCN_df = catalunya_df[catalunya_df['NMUN'] == 'Barcelona']

districtes_BCN_df

,CUMUN,CDIS,CMUN,CPRO,CUDIS,CLAU2,NPRO,NCA,NMUN,num_testigos_vc,renta_med_por_m2_vc,precio_mes_med_vc,superficie_med_vc,num_testigos_vu,renta_med_por_m2_vu,precio_mes_med_vu,superficie_med_vu
6572,8019,8,19,8,801908,8019,Barcelona,Cataluña,Barcelona,11088,11.111111,675.5000,60.0,168,8.224133,653.359836,92.0
6657,8019,7,19,8,801907,8019,Barcelona,Cataluña,Barcelona,13600,11.666667,740.1400,63.0,541,9.676564,750.000000,92.5
6718,8019,5,19,8,801905,8019,Barcelona,Cataluña,Barcelona,12951,14.004098,1100.0000,84.0,343,11.247000,1250.000000,165.0
6740,8019,9,19,8,801909,8019,Barcelona,Cataluña,Barcelona,9694,11.392405,750.4725,67.0,309,9.069444,767.111250,98.0
6763,8019,10,19,8,801910,8019,Barcelona,Cataluña,Barcelona,16817,12.326611,819.3250,69.0,264,11.518494,764.583333,71.5
6777,8019,6,19,8,801906,8019,Barcelona,Cataluña,Barcelona,12590,13.183575,858.5000,68.0,249,10.921847,910.500000,121.0
6897,8019,3,19,8,801903,8019,Barcelona,Cataluña,Barcelona,14803,12.411765,800.0000,65.0,176,9.243537,700.000000,95.0
7166,8019,2,19,8,801902,8019,Barcelona,Cataluña,Barcelona,25784,13.020750,958.5500,75.0,104,14.541627,904.475000,68.5
7689,8019,4,19,8,801904,8019,Barcelona,Cataluña,Barcelona,7042,13.717453,1000.0000,76.0,60,12.074095,1135.598527,125.0
9147,8019,1,19,8,801901,8019,Barcelona,Cataluña,Barcelona,9159,14.222222,761.9600,55.0,21,11.560284,800.000000,82.0


## 4. Model

## 5. Evaluation

## 6. Explainability